# Time Series Analysis with Polars

This notebook accompanies my blog post available here:  
https://dev.to/gaborschulz/time-series-analysis-with-polars-3dfg

The purpose is to demonstrate how you can use Polars to clean, prepare and analyze a temporal dataset. This is fairly easy with `pandas` as there are many time-related functions, and I'm curious to see if it would be similar in `polars`.

## Setup

Let's start by installing polars and configuring some of the variables required for reading the dataset.

In [9]:
!pip install --upgrade polars

In [10]:
from pathlib import Path

import polars as pl

In [11]:
# Some folder and file paths that we'll need later
BASE_PATH = Path('/content') / 'drive' / 'MyDrive' / 'time-series-polars' / 'data' # The root directory of your data files, change as needed
SOURCE_FILE =  BASE_PATH / 'nrg_105m_tabular.tsv'
INDICATORS_FILE = BASE_PATH / 'indic_nrg.csv'

# The number of lines to show when viewing a dataframe
N = 15

In [12]:
# Set the number of characters to display when showing a string in a dataframe
pl.Config.set_fmt_str_lengths(50)

polars.config.Config

## Loading the data

The dataset is a TSV file, so we'll have to make sure that Polars knows that it should split columns at Tabs, not at commas.

In [13]:
dataframe = pl.read_csv(SOURCE_FILE, separator='\t')

In [14]:
dataframe.head(n=N)

"freq,unit,product,indic_nrg,geo\TIME_PERIOD",2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,…,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""M,GWH,6000,15_107000,AT""","""5529 ""","""5098 ""","""5561 ""","""5320 ""","""5677 ""","""5822 ""","""6230 ""","""5993 ""","""5366 ""","""5411 ""","""5131 ""","""5593 ""","""5346 ""","""4933 ""","""5736 ""","""5959 ""","""6463 ""","""5933 ""","""6139 ""","""5606 ""","""5619 ""","""6175 ""","""5178 ""","""5764 ""","""6036 ""","""5434 ""","""5724 ""","""4630 ""","""5914 ""","""6461 ""","""6101 ""","""6124 ""","""6042 ""","""5913 ""","""5876 ""","""6572 """,…,"""6018 """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """
"""M,GWH,6000,15_107000,BE""","""7711 ""","""7126 ""","""6718 ""","""6334 ""","""6487 ""","""6707 ""","""7052 ""","""6810 ""","""6745 ""","""7504 ""","""7185 ""","""7765 ""","""8043 ""","""7169 ""","""7544 ""","""6815 ""","""7164 ""","""7093 ""","""7509 ""","""7841 ""","""7911 ""","""7892 ""","""7466 ""","""8336 ""","""8562 ""","""7948 ""","""8321 ""","""7543 ""","""7837 ""","""7246 ""","""7435 ""","""7702 ""","""7351 ""","""7655 ""","""7772 ""","""7761 """,…,"""7685 """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """
"""M,GWH,6000,15_107000,BG""","""4435 ""","""3678 ""","""3780 ""","""3559 ""","""3381 ""","""3268 ""","""3508 ""","""3483 ""","""3336 ""","""3608 ""","""4122 ""","""4265 ""","""4519 ""","""3892 ""","""3891 ""","""2973 ""","""2893 ""","""3082 ""","""3449 ""","""3221 ""","""3211 ""","""3523 ""","""3993 ""","""4042 ""","""4301 ""","""4036 ""","""3981 ""","""3189 ""","""2999 ""","""3576 ""","""3976 ""","""3992 ""","""3494 ""","""3889 ""","""3918 ""","""4649 """,…,"""4551 """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """
"""M,GWH,6000,15_107000,CY""","""461 ""","""415 ""","""340 ""","""323 ""","""358 ""","""443 ""","""549 ""","""539 ""","""456 ""","""372 ""","""334 ""","""401 ""","""432 ""","""382 ""","""390 ""","""321 ""","""368 ""","""470 ""","""571 ""","""546 ""","""454 ""","""412 ""","""362 ""","""424 ""","""438 ""","""393 ""","""380 ""","""342 ""","""384 ""","""448 ""","""549 ""","""617 ""","""501 ""","""402 ""","""362 ""","""391 """,…,"""430 """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": """,""": "

Here we notice a couple of issues:

1. The dataset is pivoted, i.e. months are shown as columns, which is usually not so useful for analyzing time series.
2. All columns are loaded as text. The reason for this is that whenever there is no data, there are `:` symbols in the file.
3. The name of the first column looks strange (`freq,unit,product,indic_nrg,geo\TIME_PERIOD`)

Let's clean up the dataset by fixing these issues.

## Cleaning the dataset

Let's start by unpivoting the dataframe. Polars dataframes offer the [`melt`](https://pola-rs.github.io/polars/user-guide/transformations/melt/#dataset) method to do this.

We could either list all of the columns except the id column for melting, but that would not be practical in this case. Luckily, we can provide only the id variables to the `melt` method, everything else would be considered a value variable.

In [15]:
cleaned_dataframe = dataframe.melt(id_vars='freq,unit,product,indic_nrg,geo\TIME_PERIOD', variable_name='month')
cleaned_dataframe.head(n=N)

"freq,unit,product,indic_nrg,geo\TIME_PERIOD",month,value
str,str,str
"""M,GWH,6000,15_107000,AT""","""2008-01 ""","""5529 """
"""M,GWH,6000,15_107000,BE""","""2008-01 ""","""7711 """
"""M,GWH,6000,15_107000,BG""","""2008-01 ""","""4435 """
"""M,GWH,6000,15_107000,CY""","""2008-01 ""","""461 """
"""M,GWH,6000,15_107000,CZ""","""2008-01 ""","""8497 """
"""M,GWH,6000,15_107000,DE""","""2008-01 ""","""52597 """
"""M,GWH,6000,15_107000,DK""","""2008-01 ""","""4027 """
"""M,GWH,6000,15_107000,EA""","""2008-01 ""","""212276 """
"""M,GWH,6000,15_107000,EE""","""2008-01 ""","""1125 """


If you look at the `value` column above very carefully, you'll notice that there's another, quite subtle issue: there's a redundant space at the end of each value, which would lead to an error if we tried to cast the column into a float. So we'll have to get rid of it first.

Some rows also contain letters for remarks, like `p` or `e`, so we'll have to remove those as well.

In [16]:
cleaned_dataframe = cleaned_dataframe.with_columns(
    pl.col('value')
    .str.replace('d', '')
    .str.replace('p', '')
    .str.replace('e', '')
    .str.replace(': c', ': ')
    .str.replace('n', '')
    .str.replace(': u', ': ')
    .str.strip_chars()
    .name.keep()
)
cleaned_dataframe.head(n=N)

"freq,unit,product,indic_nrg,geo\TIME_PERIOD",month,value
str,str,str
"""M,GWH,6000,15_107000,AT""","""2008-01 ""","""5529"""
"""M,GWH,6000,15_107000,BE""","""2008-01 ""","""7711"""
"""M,GWH,6000,15_107000,BG""","""2008-01 ""","""4435"""
"""M,GWH,6000,15_107000,CY""","""2008-01 ""","""461"""
"""M,GWH,6000,15_107000,CZ""","""2008-01 ""","""8497"""
"""M,GWH,6000,15_107000,DE""","""2008-01 ""","""52597"""
"""M,GWH,6000,15_107000,DK""","""2008-01 ""","""4027"""
"""M,GWH,6000,15_107000,EA""","""2008-01 ""","""212276"""
"""M,GWH,6000,15_107000,EE""","""2008-01 ""","""1125"""


Next, we'll want to get rid of the `: ` values in the `value` column and then convert the column to a numeric value. The easiest way to do that is to use the `.when().then().otherwise()` expression offered by Polars.

In [17]:
cleaned_dataframe = cleaned_dataframe.with_columns(
    pl
    .when(pl.col('value') == ':')
    .then(None)
    .otherwise(pl.col('value'))
    .name.keep()
)
cleaned_dataframe.head(n=N)

"freq,unit,product,indic_nrg,geo\TIME_PERIOD",month,value
str,str,str
"""M,GWH,6000,15_107000,AT""","""2008-01 ""","""5529"""
"""M,GWH,6000,15_107000,BE""","""2008-01 ""","""7711"""
"""M,GWH,6000,15_107000,BG""","""2008-01 ""","""4435"""
"""M,GWH,6000,15_107000,CY""","""2008-01 ""","""461"""
"""M,GWH,6000,15_107000,CZ""","""2008-01 ""","""8497"""
"""M,GWH,6000,15_107000,DE""","""2008-01 ""","""52597"""
"""M,GWH,6000,15_107000,DK""","""2008-01 ""","""4027"""
"""M,GWH,6000,15_107000,EA""","""2008-01 ""","""212276"""
"""M,GWH,6000,15_107000,EE""","""2008-01 ""","""1125"""


As you'll notice, we've replaced the missing values with a `null`. This is usually a good practice as this will help the framework identify missing values as such.

Next we'll cast the values to a numeric value.

In [18]:
cleaned_dataframe = cleaned_dataframe.with_columns(
    pl.col('value')
    .cast(pl.Float32)
    .name.keep()
)
cleaned_dataframe.head(n=N)

"freq,unit,product,indic_nrg,geo\TIME_PERIOD",month,value
str,str,f32
"""M,GWH,6000,15_107000,AT""","""2008-01 """,5529.0
"""M,GWH,6000,15_107000,BE""","""2008-01 """,7711.0
"""M,GWH,6000,15_107000,BG""","""2008-01 """,4435.0
"""M,GWH,6000,15_107000,CY""","""2008-01 """,461.0
"""M,GWH,6000,15_107000,CZ""","""2008-01 """,8497.0
"""M,GWH,6000,15_107000,DE""","""2008-01 """,52597.0
"""M,GWH,6000,15_107000,DK""","""2008-01 """,4027.0
"""M,GWH,6000,15_107000,EA""","""2008-01 """,212276.0
"""M,GWH,6000,15_107000,EE""","""2008-01 """,1125.0


Now we've cast the value column into a floating-point value. Next, let's remove the trailing string from the month and the id column as well.

In [19]:
cleaned_dataframe = cleaned_dataframe.with_columns(
    [
        pl.col('month').str.strip_chars().name.keep(),
        pl.col('freq,unit,product,indic_nrg,geo\TIME_PERIOD').str.strip_chars().name.keep()
    ]
)
cleaned_dataframe.head(n=N)

"freq,unit,product,indic_nrg,geo\TIME_PERIOD",month,value
str,str,f32
"""M,GWH,6000,15_107000,AT""","""2008-01""",5529.0
"""M,GWH,6000,15_107000,BE""","""2008-01""",7711.0
"""M,GWH,6000,15_107000,BG""","""2008-01""",4435.0
"""M,GWH,6000,15_107000,CY""","""2008-01""",461.0
"""M,GWH,6000,15_107000,CZ""","""2008-01""",8497.0
"""M,GWH,6000,15_107000,DE""","""2008-01""",52597.0
"""M,GWH,6000,15_107000,DK""","""2008-01""",4027.0
"""M,GWH,6000,15_107000,EA""","""2008-01""",212276.0
"""M,GWH,6000,15_107000,EE""","""2008-01""",1125.0


That looks much better. Now, we can focus our attention on splitting the id column into dedicated columns for each field.

In [20]:
cleaned_dataframe = (
    cleaned_dataframe
    .with_columns(
      pl.col('freq,unit,product,indic_nrg,geo\TIME_PERIOD').str.split_exact(',', 4)
    )
    .unnest('freq,unit,product,indic_nrg,geo\TIME_PERIOD')
    .rename({
        'field_0': 'freq',
        'field_1': 'unit',
        'field_2': 'product',
        'field_3': 'indic_nrg',
        'field_4': 'geo'}
    )
)
cleaned_dataframe.head(n=N)

freq,unit,product,indic_nrg,geo,month,value
str,str,str,str,str,str,f32
"""M""","""GWH""","""6000""","""15_107000""","""AT""","""2008-01""",5529.0
"""M""","""GWH""","""6000""","""15_107000""","""BE""","""2008-01""",7711.0
"""M""","""GWH""","""6000""","""15_107000""","""BG""","""2008-01""",4435.0
"""M""","""GWH""","""6000""","""15_107000""","""CY""","""2008-01""",461.0
"""M""","""GWH""","""6000""","""15_107000""","""CZ""","""2008-01""",8497.0
"""M""","""GWH""","""6000""","""15_107000""","""DE""","""2008-01""",52597.0
"""M""","""GWH""","""6000""","""15_107000""","""DK""","""2008-01""",4027.0
"""M""","""GWH""","""6000""","""15_107000""","""EA""","""2008-01""",212276.0
"""M""","""GWH""","""6000""","""15_107000""","""EE""","""2008-01""",1125.0


So, what happened here?

First, we used `.str.split_exact(',', 4)` to split the content of the id column into a structure of 5 elements.

Then, we used the `.unnest()` method on the id column to break the structure into separate columns, which were named `field_0` to `field_4` respectively.

Finally, we went on to renaming the columns to something more meaningful.

There is only one step left before we can turn out attention to exploring the data: converting the `month` column into a `date`.

In [21]:
cleaned_dataframe = cleaned_dataframe.with_columns(
    pl.col('month').str.to_date(format='%Y-%m').name.keep()
)
cleaned_dataframe.head(n=N)

freq,unit,product,indic_nrg,geo,month,value
str,str,str,str,str,date,f32
"""M""","""GWH""","""6000""","""15_107000""","""AT""",2008-01-01,5529.0
"""M""","""GWH""","""6000""","""15_107000""","""BE""",2008-01-01,7711.0
"""M""","""GWH""","""6000""","""15_107000""","""BG""",2008-01-01,4435.0
"""M""","""GWH""","""6000""","""15_107000""","""CY""",2008-01-01,461.0
"""M""","""GWH""","""6000""","""15_107000""","""CZ""",2008-01-01,8497.0
"""M""","""GWH""","""6000""","""15_107000""","""DE""",2008-01-01,52597.0
"""M""","""GWH""","""6000""","""15_107000""","""DK""",2008-01-01,4027.0
"""M""","""GWH""","""6000""","""15_107000""","""EA""",2008-01-01,212276.0
"""M""","""GWH""","""6000""","""15_107000""","""EE""",2008-01-01,1125.0


With that we've cleaned and prepared our dataset for exploratory data analysis.

## Exploring the data

Let's start by checking the frequency of each item in the columns.

In [22]:
cleaned_dataframe.select([
    pl.col('freq').value_counts(),
    pl.col('unit').value_counts(),
    pl.col('product').value_counts(),
])

freq,unit,product
struct[2],struct[2],struct[2]
"{""M"",211824}","{""GWH"",211824}","{""6000"",211824}"


As you see above, the columns `freq`, `unit` and `product` only contain 1 distinct value each, so they won't add much value to the analysis. Let's drop them.

In [23]:
cleaned_dataframe = cleaned_dataframe.drop('freq', 'unit', 'product')
cleaned_dataframe.head(n=N)

indic_nrg,geo,month,value
str,str,date,f32
"""15_107000""","""AT""",2008-01-01,5529.0
"""15_107000""","""BE""",2008-01-01,7711.0
"""15_107000""","""BG""",2008-01-01,4435.0
"""15_107000""","""CY""",2008-01-01,461.0
"""15_107000""","""CZ""",2008-01-01,8497.0
"""15_107000""","""DE""",2008-01-01,52597.0
"""15_107000""","""DK""",2008-01-01,4027.0
"""15_107000""","""EA""",2008-01-01,212276.0
"""15_107000""","""EE""",2008-01-01,1125.0


We continue looking at the frequencies of the remaining columns.

In [24]:
cleaned_dataframe.select(
    pl.col('indic_nrg').value_counts()
).unnest('indic_nrg')

indic_nrg,counts
str,u32
"""16_107105B""",5904
"""16_107101A""",5904
"""17_107302""",6192
"""B_100900""",5472
"""15_107002""",5472
"""16_107104""",6192
"""16_107105E""",5904
"""B_100300""",6192
"""15_107001""",5472


This shows us that we have 36 different indicators, with around 6,000 values per indicator.

In [25]:
cleaned_dataframe.select(
    pl.col('geo').value_counts()
).unnest('geo')

geo,counts
str,u32
"""DK""",5184
"""LU""",5184
"""SE""",5184
"""BG""",5184
"""CZ""",5184
"""SK""",5040
"""CY""",5184
"""IT""",5184
"""IE""",5184


We've got 43 different geos, with about 5000 values per geo, but with some major outliers, like `EU27_2007`, which has only 2592 values.

In [26]:
cleaned_dataframe.select(
    pl.col('month').value_counts()
).unnest('month')

month,counts
date,u32
2011-04-01,1471
2010-10-01,1471
2011-06-01,1471
2011-12-01,1471
2008-11-01,1471
2011-08-01,1471
2010-12-01,1471
2008-02-01,1471
2009-01-01,1471


We've got a total of 144 months, i.e. 12 years in the dataset.

In [27]:
cleaned_dataframe.describe()

describe,indic_nrg,geo,month,value
str,str,str,str,f64
"""count""","""211824""","""211824""","""211824""",211824.0
"""null_count""","""0""","""0""","""0""",108232.0
"""mean""",null,null,null,6298.541992
"""std""",null,null,null,26036.425781
"""min""","""15_107000""","""AL""","""2008-01-01""",-8166.0
"""25%""",null,null,null,0.0
"""50%""",null,null,null,282.725006
"""75%""",null,null,null,1910.0
"""max""","""B_107200""","""UK""","""2019-12-01""",310240.0


The describe statistics show us that we've got a total of 211,824 values in the dataset, out of which 108,232 are `null`, i.e. missing values.

We saw above that the `indic_nrg` field is quite hard to read, because it's only a code of the indicator. It would be great if we could show the name of the indicator as well.

Let's load the list of indicators for the `indic_nrg.csv` file.

In [28]:
indicators = pl.read_csv(INDICATORS_FILE)
indicators.head(n=N)

indicator_name,id
str,str
"""imports""","""B_100300"""
"""imports_from_eu""","""B_100320"""
"""exports""","""B_100500"""
"""exports_to_EU""","""B_100510"""
"""gross_inland_consumption""","""B_100900"""
"""Used for pumped storage""","""17_107302"""
"""distribution_losses""","""B_101400"""
"""available_for_internal_market""","""B_107200"""
"""gross_electricity_generation_total""","""15_107000"""


We have way too many indicators to analyze in a meaningful way, so we should limit our attention and keep only a few of them. I've preapred the list of indicators in a way that all the indicators we want to keep contain underscores, and the ones we want to drop do not contain any.

This makes our life easier when we want to remove the unneeded indicators.

In [29]:
indicators = indicators.filter(pl.col('indicator_name').str.contains('_'))
indicators.head(n=N)

indicator_name,id
str,str
"""imports_from_eu""","""B_100320"""
"""exports_to_EU""","""B_100510"""
"""gross_inland_consumption""","""B_100900"""
"""distribution_losses""","""B_101400"""
"""available_for_internal_market""","""B_107200"""
"""gross_electricity_generation_total""","""15_107000"""
"""gross_electricity_generation_hydro""","""15_107001"""
"""gross_electricity_generation_geothermal""","""15_107002"""
"""gross_electricity_generation_nuclear""","""15_107003"""


In the next step, we're going to join the indicators dataframe with the cleaned dataset. We're going to do an inner join, which will eliminate the indicators not present in the `indicators` dataframe. This is a very efficient way to get rid of rows we don't need.

In [30]:
cleaned_dataframe = cleaned_dataframe.join(indicators, left_on='indic_nrg', right_on='id')
cleaned_dataframe.head(n=N)

indic_nrg,geo,month,value,indicator_name
str,str,date,f32,str
"""15_107000""","""AT""",2008-01-01,5529.0,"""gross_electricity_generation_total"""
"""15_107000""","""BE""",2008-01-01,7711.0,"""gross_electricity_generation_total"""
"""15_107000""","""BG""",2008-01-01,4435.0,"""gross_electricity_generation_total"""
"""15_107000""","""CY""",2008-01-01,461.0,"""gross_electricity_generation_total"""
"""15_107000""","""CZ""",2008-01-01,8497.0,"""gross_electricity_generation_total"""
"""15_107000""","""DE""",2008-01-01,52597.0,"""gross_electricity_generation_total"""
"""15_107000""","""DK""",2008-01-01,4027.0,"""gross_electricity_generation_total"""
"""15_107000""","""EA""",2008-01-01,212276.0,"""gross_electricity_generation_total"""
"""15_107000""","""EE""",2008-01-01,1125.0,"""gross_electricity_generation_total"""


The number of rows dropeed from 211,824 to only 62,064.

It would be great to move the indicators into the columns, as they are related to the same geo and the same month. This procedure is called pivoting.

In [31]:
pivoted_dataframe = cleaned_dataframe.pivot(values='value', index=['geo', 'month'], columns='indicator_name', aggregate_function='sum')
pivoted_dataframe.head(n=N)

geo,month,gross_electricity_generation_total,gross_electricity_generation_hydro,gross_electricity_generation_geothermal,gross_electricity_generation_nuclear,gross_electricity_generation_conventional_thermal,gross_electricity_generation_wind,imports_from_eu,exports_to_EU,gross_inland_consumption,distribution_losses,available_for_internal_market
str,date,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""AT""",2008-01-01,5529.0,3050.0,0.0,0.0,2479.0,0.0,0.0,0.0,1109.0,0.0,6187.0
"""BE""",2008-01-01,7711.0,261.0,0.0,4067.0,3383.0,0.0,0.0,0.0,1140.0,0.0,8400.0
"""BG""",2008-01-01,4435.0,345.0,0.0,1535.0,2555.0,0.0,0.0,0.0,-107.0,0.0,3808.0
"""CY""",2008-01-01,461.0,0.0,0.0,0.0,461.0,0.0,0.0,0.0,0.0,0.0,436.0
"""CZ""",2008-01-01,8497.0,266.0,0.0,2820.0,5411.0,0.0,0.0,0.0,-1467.0,0.0,6352.0
"""DE""",2008-01-01,52597.0,2265.0,0.0,12976.0,37356.0,0.0,0.0,0.0,-3893.0,0.0,44608.0
"""DK""",2008-01-01,4027.0,1054.0,0.0,0.0,2973.0,0.0,0.0,0.0,-432.0,0.0,3430.0
"""EA""",2008-01-01,212276.0,23365.0,486.0,69959.0,118466.0,0.0,null,null,1009.0,null,199686.0
"""EE""",2008-01-01,1125.0,18.0,0.0,0.0,1107.0,0.0,0.0,0.0,-150.0,0.0,864.0


We're left with a dataframe with 6,192 rows and 13 columns. We can now start asking questions about our data.

### Questions

**Question #1**  

How much energy did each country generate in total in the timeframe?

To answer this question, we're going to group the data per geo and sum up the total energy generated. We'll sort them in descending order for convenience.

In [32]:
(
    pivoted_dataframe
    .group_by('geo')
    .agg(pl.col('gross_electricity_generation_total').sum())
    .sort('gross_electricity_generation_total', descending=True)
)

geo,gross_electricity_generation_total
str,f32
"""EU27_2007""",2.8242368e7
"""EA""",1.973906e7
"""EU28""",1.2451493e7
"""EU27_2020""",1.1085141e7
"""FR""",5.05411e6
"""DE""",4.865313e6
"""UK""",3.245098e6
"""IT""",2.634892e6
"""ES""",2.501921e6


**Answer #1**  

The table above shows how much energy each geo generated. The highest amount was generated by France, followed by Germany and the UK. Some countries have no data at all, thus their values are shown to be 0.


**Question #2**  
How much energy did France generate each year?

To figure this out, we need to do something called [downsampling](https://machinelearningmastery.com/resample-interpolate-time-series-data-python/). Downsampling reduces the frequency, e.g. from months to years.

In Polars, downsampling is really easy, it's simply a special case of grouping, done with the `group_by_dynamic` method. This method has a special requirement: the data has to be sorted by the column which contains the date (`month` in our case).

In [33]:
(
    pivoted_dataframe
    .filter(pl.col('geo') == 'FR')
    .sort('month')
    .group_by_dynamic('month', every='1y')
    .agg(pl.col('gross_electricity_generation_total').sum())
)

month,gross_electricity_generation_total
date,f32
2008-01-01,573519.0
2009-01-01,542390.0
2010-01-01,572960.0
2011-01-01,564607.0
2012-01-01,560538.0
2013-01-01,561078.0
2014-01-01,557514.0
2015-01-01,568045.0
2016-01-01,553459.0


**Answer #2**  

The table above shows us the energy generated by France each year. We can see a peak in 2008 and a 2nd peak in 2010.


**Question #3**

Which country generated the highest proportion of its energy mix from renewable sources each year?

We're going to define renewable energies as hydro, geothermal and wind for the purpose of this analysis.

First, we'll have to create a new column for renewable sources. Then, we're going to aggregate the data up to an annual value and figure out which country did best each year.

In [34]:
(
    pivoted_dataframe
    .select(
        pl.col('geo'),
        pl.col('month'),
        pl.col('gross_electricity_generation_total'),
        (
            pl.col('gross_electricity_generation_hydro') +
            pl.col('gross_electricity_generation_geothermal') +
            pl.col('gross_electricity_generation_wind')
        ).alias('gross_electricity_generation_renewable'),
    )
    .sort('month')
    .group_by_dynamic('month', by='geo', every='1y')
    .agg(pl.col('gross_electricity_generation_total', 'gross_electricity_generation_renewable').sum())
    .select(
        pl.col('geo'),
        pl.col('month'),
        (pl.col('gross_electricity_generation_renewable') / pl.col('gross_electricity_generation_total') * 100).alias('renewable_pct')
    )
    .with_columns(
        pl.col('renewable_pct').max().over('month').alias('renewable_pct_max')
    )
    .filter(
        pl.col('renewable_pct') == pl.col('renewable_pct_max')
    )
    .sort('month')
    .select('month', 'geo', 'renewable_pct')
)

month,geo,renewable_pct
date,str,f32
2008-01-01,"""NO""",99.169304
2009-01-01,"""NO""",97.202293
2010-01-01,"""NO""",95.778259
2011-01-01,"""NO""",96.251648
2012-01-01,"""IS""",99.966034
2013-01-01,"""IS""",99.989006
2014-01-01,"""IS""",100.016617
2015-01-01,"""IS""",99.958076
2016-01-01,"""NO""",97.677307


**Answer #3**  
This shows us that Norway and Iceland were continously competing with each other for the first place, as both countries essentially generated almost 100% of their electricity from renewable sources.

The above expression looks quite hairy, so let's walk through each line one-by-one.

`Line 1: (`  
I typically like to break such expressions into multiple lines for better readability. Starting with a bracket helps me avoid having to put a `\` at the end of each line, further improving readability

`Line 2: pivoted_dataframe`  
This is quite self-explanatory, we start off by taking the dataframe we built before.

`Line 3: select(`  
We want to pick a couple of columns and build our new dataframe from them. The `select` method is made right for that purpose.

`Line 4 - 6: pl.col(...)`  
We pick the `geo`, the `month` and the `gross_electricity_generation_total` column as they are, without doing anything special with them.

`Line 7 - 12`  
We're adding up the hydro, geothermal and wind power per month and per geo, and calling the sum `gross_electricity_generation_renewable` using the `.alias` method.

`Line 14: sort('month')`  
As we figured out before, we need to sort the data by the `month` for the `group_by_dynamic` to work as expected.

`Line 15: .group_by_dynamic('month', by='geo', every='1y')`  
We are using the `month` column again to create aggergations on an annual level, but this time, we're adding the `by='geo'` argument, which let's us define which other columns we want to group by as well.

`Line 16: .agg(pl.col('gross_electricity_generation_total', 'gross_electricity_generation_renewable').sum())`  
After grouping values on line 15, we're performing the aggregation on this line for the total and the renewable column.

`Line 17 - 20: select(...)`  
We're selecting the geo and the month column, and we use the `gross_electricity_generation_renewable` and `gross_electricity_generation_total` column to calculate the percentage per geo and per year.

`Line 21 - 23: with_columns(...)`  
We're calculating the maximum percentage of renewables. We use the `.over()` method to calculate the maximum percentage for each date entry in the dataset.

`Line 24 - 26: filter(...)`  
We're checking if a given line is equal to that year's maximum value. If it is, then we've found the geo with the maximum percentage for the given year.

`Line 27: .sort('month')`  
This is optional, but it makes it easier to read if we sort the values in ascending order by date.

`Line 28: .select('month', 'geo', 'renewable_pct')`  
We don't need the `renewable_pct_max` column any more, so we can either select the columns we want (like we did here), or we could simply drop the columns we don't need any more.

`Line 29: )`  
We close the parenthesis we opened on line 1.

## Conclusion

It turned out that polars offers a very easy API for working with time-based datasets for both cleaning and exploring the data. Pairing this with the ability to work on multiple CPUs, process datasets that are larger than the computer's memory thanks to lazy calculation, I believe that polars is an amazing tool for every data scientist's and analyst's toolbelt.

**Go Polars!!!**